# Battery Lifetime Calculator

## How to use this document

Click the run button on each block to step through the calculations.   

Some of the blocks have values that you can change before you click run.

## What does this document do
This calculation turns battery capacity around, and asks, you have a device that does some job, how many jobs can you do?.   

Each job uses high power, but the device also uses a smaller amount of power all the time (its *standby consumption*).

How many jobs can you do in a day, before you need to recharge?

Battery capacity is measured in Ampere-Hours, that is Amps multiplied by Hours.   Its a literal trade-off---draw a lot of amps, you only get a few hours; draw a fraction of an amp, and you get lots of hours.  A 10Ah battery can theoretically deliver 10 Amps for 1 hour, one Amp for 10 hours, or 500mA for 20 hours.   In reality, the relationship might not be so *linear*.

The rated capacity from the manufacturer is usually for a 20 hour discharge, so if you are going to run the battery for a shorter or longer time, you may experience a different effective capacity.    Also, resellers lie outrageously about capacity.

If your load is constant (eg you draw 200mA all the time, then a theoretical lifetime is simply `capacity / current`, eg `10Ah / 200mA => 50 hours`.

Since we are considering a 24 hour discharge, we are close to the manufacturer figure, but practically 
we are using different rates, our gadget will be in standby (sleep mode) when possible and only consume our **peak draw** when doing a job.  The rest of the time we consume a far smaller **standby draw**

We'll use Python and the *Pint* unit library to help us model this scenario.

In [60]:
# Load some library modules.   You can ignore this block, just click run.
import pint
units = pint.UnitRegistry()
from math import floor

## Enter your battery capacity

The rated capacity presumes you use all the power in the battery.   Running many kinds of battery dead flat can damage them, so lets choose a safety factor.   We'll say we only run the battery 80% flat.



In [61]:
# Edit the values in this block below and then click "Run".

rated_capacity = 120 * units.Ah

depth_of_discharge = 0.8

# Don't change anything below here in this block
capacity = rated_capacity * depth_of_discharge
print("If we are nice to our battery we can use up to {} of its rated {} capacity".format(capacity,rated_capacity))

If we are nice to our battery we can use up to 96.0 ampere_hour of its rated 120 ampere_hour capacity


## Describe your power needs

### Enter your job power consumption   

How much power do you need when doing a job?


In [62]:
# Edit the values in this block below and then click "Run".

job_draw = 1500 * units.mA

job_duration = 15 * units.minute

# Don't change anything below here in this block
job_energy_use = (job_draw * job_duration).to(units.Ah);
print("At {} for {} we use {} per job".format(job_draw, job_duration, job_energy_use))

At 1500 milliampere for 15 minute we use 0.375 ampere_hour per job


### Enter your standby consumption   

How much power do you consume when in standby mode?

Let's take a shortcut and presume we use the standby energy even during jobs.  If we didn't take this shortcut we would need to solve a set of simultaneous equations.


In [67]:

standby_draw = 200 * units.mA

# Don't change anything below here in this block
standby_energy_use = (standby_draw * 24 * units.hour).to(units.Ah)
print("At {0} standby, our gadget is going to consume {1:.5} each day even when idle.".format(standby_draw,standby_energy_use))
print("That's {}% of our effective capacity just for standby.".format(floor(100*standby_energy_use/capacity)))

At 200 milliampere standby, our gadget is going to consume 4.8 ampere_hour each day even when idle.
That's 4% of our effective capacity just for standby.


## Caculate the theoretical job capacity

Now we know how much energy we use per day just when idle, we can divide the remainder by 24h to work out how many jobs we could do per day in the best case.    

But we don't work 24 hours.   We have a "working" day, so lets work ou

In reality, there is efficiency, non-linearity and self-discharge to consider, so this figure may not be precise.

In [80]:
# Edit the values in this block below and then click "Run".

working_hours_per_day = 10 * units.hour

# You don't need to edit this block, just click "Run" to see the calculated value
job_energy_budget = capacity - standby_energy_use
job_capacity = floor(job_energy_budget / job_energy_use)
print("In theory, we can do up to {} x {} jobs, per day.".format(job_capacity,job_duration))
user_capacity = floor(job_capacity * job_duration / working_hours_per_day)
print("That means that in a {} working day, we can serve {} users at a time".format(working_hours_per_day,user_capacity))

In theory, we can do up to 243 x 15 minute jobs, per day.
That means that in a 10 hour working day, we can serve 6 users at a time


**Note**: In theory, theory and practice are the same thing.  In practice, they are not.